In [2]:
import numpy as np
import dask.dataframe as dd
import dask.array as da
from dask.diagnostics import ProgressBar 
import dask.delayed as delayed
from matplotlib import pyplot as plt 
%matplotlib inline

In [5]:
from gcsfs import GCSFileSystem
gcs = GCSFileSystem(project='spry-equator-289113')
gcs.ls('ac295bucket1')

['ac295bucket1/Parking_Violations_Issued_-_Fiscal_Year_2017.csv']

In [6]:
from distributed import Client
# client = Client(n_workers=4,
#     processes=True,
#     threads_per_worker=1,memory_limit='2GB')
# client
client = Client()
client

/opt/conda/lib/python3.8/site-packages/distributed/client.py:1130: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| python  | 3.8.5.final.0 | 3.8.0.final.0 | 3.8.0.final.0 |
+---------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tcp://my-release-dask-scheduler:8786 Dashboard: http://my-release-dask-scheduler:8787/status,Cluster Workers: 3 Cores: 3 Memory: 11.63 GB


In [7]:
# to avoid dtype warning
dict1 = {'Summons Number': 'int64',  'Plate ID': 'object', 'Registration State': 'object', 'Plate Type': 'object',
 'Issue Date': 'object', 'Violation Code': 'int64', 'Vehicle Body Type': 'object', 'Vehicle Make': 'object',
 'Issuing Agency': 'object', 'Street Code1': 'int64', 'Street Code2': 'int64', 'Street Code3': 'int64',
 'Vehicle Expiration Date': 'int64', 'Violation Location': 'float64', 'Violation Precinct': 'int64', 'Issuer Precinct': 'int64',
 'Issuer Code': 'int64', 'Issuer Command': 'object', 'Issuer Squad': 'object', 'Violation Time': 'object',
 'Time First Observed': 'object', 'Violation County': 'object', 'Violation In Front Of Or Opposite': 'object', 'House Number': 'object',
 'Street Name': 'object', 'Intersecting Street': 'object', 'Date First Observed': 'int64', 'Law Section': 'int64',
 'Sub Division': 'object', 'Violation Legal Code': 'object', 'Days Parking In Effect    ': 'object', 'From Hours In Effect': 'object',
 'To Hours In Effect': 'object', 'Vehicle Color': 'object', 'Unregistered Vehicle?': 'float64', 'Vehicle Year': 'int64',
 'Meter Number': 'object', 'Feet From Curb': 'int64', 'Violation Post Code': 'object', 'Violation Description': 'object',
 'No Standing or Stopping Violation': 'float64', 'Hydrant Violation': 'float64', 'Double Parking Violation': 'float64'}


In [8]:
import time 
start = time.time()

df = dd.read_csv('gs://ac295bucket1/Parking_Violations_Issued_-_Fiscal_Year_2017.csv',
                 storage_options={'token': 'anon'}, 
                 dtype=dict1)

# Listing 2.2 Counting missing values in the DataFrame
missing_values = df.isnull().sum()

# Listing 2.3  Calculating the percent of missing values in the DataFrame
missing_count = ((missing_values / df.index.size) * 100)


# Listing 2.4 - Computing the DAG
# with ProgressBar(): 
missing_count_pct = missing_count.compute()
missing_count_pct

_call non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/gcsfs/core.py", line 507, in _call
    self.validate_response(status, contents, json, path, headers)
  File "/opt/conda/lib/python3.8/site-packages/gcsfs/core.py", line 1228, in validate_response
    raise HttpError(error)
gcsfs.utils.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.


HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.

In [7]:
time.time() - start

68.29123044013977

In [8]:
# Listing 2.5 - Filtering sparse columns

columns_to_drop = missing_count_pct[missing_count_pct > 60].index
df_dropped = df.drop(columns_to_drop, axis=1).persist() 

In [9]:
df_dropped

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Date First Observed,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Vehicle Year,Feet From Curb,Violation Post Code,Violation Description
npartitions=33,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,int64,object,object,object,int64,int64,int64,int64,float64,int64,int64,int64,object,object,object,object,object,object,object,int64,int64,object,object,object,object,object,int64,int64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
